## 1. Arrangements

In [2]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import StructType,StructField,DoubleType, StringType, IntegerType, DateType, ArrayType, FloatType, LongType, TimestampType


In [3]:
spark = SparkSession.builder \
.master("local[2]") \
.appName("Define Custom Schema") \
.getOrCreate()

2023-05-13 14:41:05,819 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Download the Dataset

In [5]:
#! wget -P /home/train/datasets/ \
# https://github.com/erkansirin78/datasets/raw/master/sensors_instrumented_in_an_office_building_dataset.zip

--2023-05-13 14:43:57--  https://github.com/erkansirin78/datasets/raw/master/sensors_instrumented_in_an_office_building_dataset.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/erkansirin78/datasets/master/sensors_instrumented_in_an_office_building_dataset.zip [following]
--2023-05-13 14:43:57--  https://raw.githubusercontent.com/erkansirin78/datasets/master/sensors_instrumented_in_an_office_building_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 78299066 (75M) [application/zip]
Saving to: ‘/home/train/datasets/sensors_instrumented_in_an_office_building_dataset.zip’

100%[=======

### Reading Data

In [ ]:
programmatical_schema = StructType([
    StructField("ts_min_bignt",LongType(),True),
    StructField("value",FloatType(),True),
    StructField("file_name",StringType(),True),
    StructField("room",StringType(),True),
    StructField("file_path",StringType(),True),
    StructField("file_name_type",StringType(),True),
    StructField("event_ts_min",TimestampType(),True),
])
    

In [24]:
df = spark.read \
.option("sep",",") \
.schema(programmatical_schema) \
.csv("file:///home/train/datasets/KETI/*") \
.withColumn("file_path",F.input_file_name()) \
.withColumn("room",F.element_at(F.reverse(F.split(F.col("file_path"),"/")),2)) \
.withColumn("file_name",F.element_at(F.reverse(F.split(F.col("file_path"),"/")),1)) \
.withColumn("file_name_type",F.split(F.col("file_name"),"\\.").getItem(0)) \
.filter("file_name_type not in ('README')") \
.withColumn("event_ts_min",F.from_unixtime(F.col("ts_min_bignt")))

In [25]:
df.limit(10).toPandas()

,ts_min_bignt,value,file_name,room,file_path,file_name_type,event_ts_min
0,1377299103,2287.0,light.csv,668,file:/home/train/datasets/KETI/668/light.csv,light,2013-08-24 02:05:03
1,1377299105,1977.0,light.csv,668,file:/home/train/datasets/KETI/668/light.csv,light,2013-08-24 02:05:05
2,1377299126,2208.0,light.csv,668,file:/home/train/datasets/KETI/668/light.csv,light,2013-08-24 02:05:26
3,1377299130,2267.0,light.csv,668,file:/home/train/datasets/KETI/668/light.csv,light,2013-08-24 02:05:30
4,1377299133,2097.0,light.csv,668,file:/home/train/datasets/KETI/668/light.csv,light,2013-08-24 02:05:33
5,1377299138,2111.0,light.csv,668,file:/home/train/datasets/KETI/668/light.csv,light,2013-08-24 02:05:38
6,1377299143,2257.0,light.csv,668,file:/home/train/datasets/KETI/668/light.csv,light,2013-08-24 02:05:43
7,1377299148,2200.0,light.csv,668,file:/home/train/datasets/KETI/668/light.csv,light,2013-08-24 02:05:48
8,1377299153,2164.0,light.csv,668,file:/home/train/datasets/KETI/668/light.csv,light,2013-08-24 02:05:53
9,1377299158,2183.0,light.csv,668,file:/home/train/datasets/KETI/668/light.csv,light,2013-08-24 02:05:58


In [14]:
df.count()

29882394

In [16]:
df.selectExpr("MAX(event_ts_min)", "MIN(event_ts_min)").limit(5).toPandas()

,max(event_ts_min),min(event_ts_min)
0,2013-09-01 09:58:59,2013-08-23 18:00:00


In [35]:
#1er Dakikalık groupby yaparak ortalamalara göre valueları al
df2= df.groupby("file_name_type","room",F.window("event_ts_min","1 minutes")) \
.agg(F.avg("value").alias("value")) \
.select(F.to_timestamp(F.col("window.start"), 'MM/dd/yyy HH:mm:ss a').alias("event_ts_min"),F.unix_timestamp(F.col("window.start"), 'MM/dd/yyyy hh:mm:ss a').alias("ts_min_bignt"), F.col("file_name_type"),F.col("room"),F.col("value"))

In [36]:
df2.limit(10).toPandas()

,event_ts_min,ts_min_bignt,file_name_type,room,value
0,2013-08-24 07:23:00,1377318180,light,668,2137.916667
1,2013-08-24 07:28:00,1377318480,light,668,2212.416667
2,2013-08-24 08:17:00,1377321420,light,668,2135.333333
3,2013-08-24 12:11:00,1377335460,light,668,2172.250000
4,2013-08-24 12:45:00,1377337500,light,668,2228.166667
5,2013-08-24 15:34:00,1377347640,light,668,2186.083333
6,2013-08-24 16:23:00,1377350580,light,668,2142.916667
7,2013-08-24 17:42:00,1377355320,light,668,2224.750000
8,2013-08-24 19:44:00,1377362640,light,668,2185.916667
9,2013-08-24 20:57:00,1377367020,light,668,2281.500000


In [ ]:
# pivot ile filename typeı sütun olarak al 
df3 = df2.groupby('event_ts_min','ts_min_bignt','room').pivot("file_name_type").mean("value")

In [38]:
df3.limit(20).toPandas()

,event_ts_min,ts_min_bignt,room,co2,humidity,light,pir,temperature
0,2013-08-26 12:32:00,1377509520,562,238.083333,61.282499,2.416667,0.0,21.912500
1,2013-08-30 12:22:00,1377854520,776,518.250000,58.419999,4.400000,0.0,23.419000
2,2013-08-31 01:12:00,1377900720,722,437.250000,55.070001,4.916667,0.0,23.231666
3,2013-08-28 08:40:00,1377668400,415,484.250000,57.557501,3.750000,0.0,23.237500
4,2013-08-28 08:34:00,1377668040,746,514.833333,60.747501,5.000000,0.0,22.293333
5,2013-08-26 03:10:00,1377475800,562,227.583333,63.600000,3.416667,0.0,22.054166
6,2013-08-30 20:29:00,1377883740,717,444.000000,61.295000,124.000000,0.0,22.504167
7,2013-08-26 15:43:00,1377520980,734,291.250000,54.875000,5.583333,0.0,23.279167
8,2013-08-26 14:18:00,1377515880,562,232.000000,60.862501,2.416667,0.0,21.916667
9,2013-08-28 10:53:00,1377676380,448,362.500000,57.805001,4.416667,0.0,23.288334


In [39]:
df3.count()

602667

### Clear Null Values

In [40]:
df3.where("co2 is null").show()

+-------------------+------------+----+----+-----------------+------------------+-------+------------------+
|       event_ts_min|ts_min_bignt|room| co2|         humidity|             light|    pir|       temperature|
+-------------------+------------+----+----+-----------------+------------------+-------+------------------+
|2013-08-28 01:31:00|  1377642660| 417|null|             null|              null|    0.0|              null|
|2013-09-01 08:58:00|  1378015080| 562|null|             null|              null|    0.0|              null|
|2013-09-01 05:44:00|  1378003440| 648|null|             null|              null|    0.0|              null|
|2013-09-01 04:25:00|  1377998700| 721|null|             null|              null|    0.0|              null|
|2013-09-01 03:13:00|  1377994380| 664|null|             null|              null|    0.0|              null|
|2013-09-01 01:50:00|  1377989400| 454|null|             null|              null|    0.0|              null|
|2013-08-31 22:35:0

In [41]:
df_test= df3.where("ts_min_bignt == '1377299100' and room == '413' ")

In [42]:
df_test.limit(5).toPandas()

,event_ts_min,ts_min_bignt,room,co2,humidity,light,pir,temperature
0,2013-08-24 02:05:00,1377299100,413,494.727273,45.330001,96.555556,0.0,23.927778


In [43]:
df3.printSchema()

root
 |-- event_ts_min: timestamp (nullable = true)
 |-- ts_min_bignt: long (nullable = true)
 |-- room: string (nullable = true)
 |-- co2: double (nullable = true)
 |-- humidity: double (nullable = true)
 |-- light: double (nullable = true)
 |-- pir: double (nullable = true)
 |-- temperature: double (nullable = true)



In [44]:
df4 = df3.select(F.col("event_ts_min"),
                 F.col("ts_min_bignt"),
                 F.col("room"),
                 F.coalesce(F.col("co2"),F.lit(0)).alias("co2"),
                 F.coalesce(F.col("light"),F.lit(0)).alias("light"),
                 F.coalesce(F.col("temperature"),F.lit(0)).alias("temperature"),
                 F.coalesce(F.col("humidity"),F.lit(0)).alias("humidity"),
                 F.coalesce(F.col("pir"),F.lit(0)).alias("pir")
                )
                 

In [45]:
df4.printSchema()

root
 |-- event_ts_min: timestamp (nullable = true)
 |-- ts_min_bignt: long (nullable = true)
 |-- room: string (nullable = true)
 |-- co2: double (nullable = false)
 |-- light: double (nullable = false)
 |-- temperature: double (nullable = false)
 |-- humidity: double (nullable = false)
 |-- pir: double (nullable = false)



In [46]:
df4.count()

602667

In [47]:
df4.selectExpr("MAX(event_ts_min)", "MIN(event_ts_min)").limit(5).toPandas()

,max(event_ts_min),min(event_ts_min)
0,2013-09-01 09:58:00,2013-08-23 18:00:00


In [48]:
df_test2= df4.where("ts_min_bignt == '1377299100' and room == '413' ")

In [49]:
df_test2.limit(5).toPandas()

,event_ts_min,ts_min_bignt,room,co2,light,temperature,humidity,pir
0,2013-08-24 02:05:00,1377299100,413,494.727273,96.555556,23.927778,45.330001,0.0


### Write as csv file

In [50]:
df4.repartition(1).write.csv("file:///home/train/project_sensors_output/",header=True)

### Check the values

In [52]:
df_source = spark.read \
.format("csv") \
.option("sep",",") \
.option("header",True) \
.option("inferSchema",True) \
.load("file:///home/train/project_sensors_output/part-00000-a2de2d7d-e0dd-4ecd-a201-a783d8d55f40-c000.csv")


In [51]:
programmaticall_schema_2 = StructType([
    StructField("event_ts_min",TimestampType(),True),
    StructField("ts_min_bignt",LongType(),True),
    StructField("room",StringType(),True),
    StructField("co2",FloatType(),True),
    StructField("light",FloatType(),True),
    StructField("temp",FloatType(),True),
    StructField("humidity",FloatType(),True),
    StructField("pir",FloatType(),True)
])

In [53]:
df_source.limit(30).toPandas()

,event_ts_min,ts_min_bignt,room,co2,light,temperature,humidity,pir
0,2013-08-26 12:32:00,1377509520,562,238.083333,2.416667,21.912500,61.282499,0.0
1,2013-08-30 12:22:00,1377854520,776,518.250000,4.400000,23.419000,58.419999,0.0
2,2013-08-31 01:12:00,1377900720,722,437.250000,4.916667,23.231666,55.070001,0.0
3,2013-08-28 08:40:00,1377668400,415,484.250000,3.750000,23.237500,57.557501,0.0
4,2013-08-28 08:34:00,1377668040,746,514.833333,5.000000,22.293333,60.747501,0.0
5,2013-08-26 03:10:00,1377475800,562,227.583333,3.416667,22.054166,63.600000,0.0
6,2013-08-30 20:29:00,1377883740,717,444.000000,124.000000,22.504167,61.295000,0.0
7,2013-08-26 15:43:00,1377520980,734,291.250000,5.583333,23.279167,54.875000,0.0
8,2013-08-26 14:18:00,1377515880,562,232.000000,2.416667,21.916667,60.862501,0.0
9,2013-08-28 10:53:00,1377676380,448,362.500000,4.416667,23.288334,57.805001,0.0


In [54]:
df_source.count()

602667

In [55]:
df_test3= df_source.where("ts_min_bignt == '1377299100' and room == '413'")

In [56]:
df_test3.toPandas()

,event_ts_min,ts_min_bignt,room,co2,light,temperature,humidity,pir
0,2013-08-24 02:05:00,1377299100,413,494.727273,96.555556,23.927778,45.330001,0.0


2023-05-14 14:14:22,839 WARN spark.HeartbeatReceiver: Removing executor driver with no recent heartbeats: 36619675 ms exceeds timeout 120000 ms
2023-05-14 14:14:22,852 WARN spark.SparkContext: Killing executors is not supported by current scheduler.
